<a href="https://colab.research.google.com/github/Droppicode/classify_transactions/blob/main/classify_transactions_make_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import random
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)

# Abre base de dados

with open("base_dados.json", "r") as json_file:
    base_dados = json.load(json_file)

categorias = list(base_dados.keys())

# Sujeira para as transacoes

sujeira_prefixos = ['COMPRA', 'PGTO', 'DEBITO', 'CREDITO', 'PIX', 'TED', 'DOC', 'EXTRATO', 'COMPRA ELO', 'COMPRA VISA', 'ELO', 'VISTA']
sujeira_sufixos = ['SP', 'RJ', 'BH', 'CURITIBA', 'MATRIZ', 'FILIAL', 'S.A.', 'LTDA', 'PAGAMENTOS']

def gerar_dataset(qtd_linhas=5000):
    transacoes = []

    for _ in range(qtd_linhas):
        # Escolhe uma loja aleatória
        cat = random.choice(categorias)
        loja = random.choice(base_dados[cat])

        # Gera a descrição suja
        desc = loja
        if random.random() < 0.6: desc = f"{random.choice(sujeira_prefixos)} {desc}"
        if random.random() < 0.4: desc = f"{desc} {random.choice(sujeira_sufixos)}"
        if random.random() < 0.3: desc = f"{desc} {random.randint(10, 9999)}"

        transacoes.append([desc, cat])

    return pd.DataFrame(transacoes, columns=['Descricao', 'Categoria'])

# Executa e salva

print("Gerando 10.000 transações...")
df = gerar_dataset(10000)
df.to_csv('transacoes_train.csv', index=False)

print("Amostra do resultado:")
df

Gerando 10.000 transações...
Amostra do resultado:


,Descricao,Categoria
0,GARAGEM 198,MANUTENCAO
1,PAGAMENTO IPVA 484,IMPOSTOS
2,COMPRA VISA MINISO SP,CASA
3,DOC LOJAS POMPEIA PAGAMENTOS,LOJAS DE DEPARTAMENTO
4,FOGAS,GAS
...,...,...
9995,AGUAS DE NITEROI LTDA,AGUA
9996,FERREIRA COSTA BH 2839,CASA
9997,ISS BH 7650,IMPOSTOS
9998,PIX FGTS,IMPOSTOS


In [54]:
df = df[~df['Descricao'].str.contains(r'\bPIX\b', regex=True)]
df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
for w in sujeira_prefixos + sujeira_sufixos:
    df['Descricao'] = df['Descricao'].str.replace(w, '')

df['Categoria'] = df['Categoria'].str.upper().str.strip()

/tmp/ipython-input-1018725854.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
/tmp/ipython-input-1018725854.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Descricao'] = df['Descricao'].str.replace(w, '')
/tmp/ipython-input-1018725854.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df['Descricao']
y = df['Categoria']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

model = make_pipeline(
    TfidfVectorizer(),
    RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
)

print("Treinando modelos para todas as colunas...")
model.fit(X_train, y_train)
print("Treino concluído!")

Treinando modelos para todas as colunas...
Treino concluído!


In [56]:
y_pred = model.predict(X_test)

df_final = pd.DataFrame({
    'Descricao': X_test,
    'Real': y_test,
    'Previsto': y_pred
})

probas = model.predict_proba(X_test)
max_probas = np.max(probas, axis=1)
df_final['Confianca'] = max_probas

print(df_final.head(10))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

                     Descricao                   Real               Previsto  Confianca
1471                FAST SHOP                    CASA                   CASA   0.572215
1576                     SOUQ                    CASA                   CASA   0.665049
9208          DROGARIA NISSEI            MEDICAMENTOS           MEDICAMENTOS   1.000000
2628               CASA RIACHU  LOJAS DE DEPARTAMENTO  LOJAS DE DEPARTAMENTO   1.000000
3788                   HOTMART               EDUCACAO               EDUCACAO   0.990000
432           ANHEMBI MORUMBI                EDUCACAO               EDUCACAO   0.980000
4971                 KINOPLEX                   LAZER                  LAZER   1.000000
9658   VISA EQUATORIAL ENERGIA                    LUZ                    LUZ   1.000000
8326                   NETFLIX            ASSINATURAS            ASSINATURAS   1.000000
3023   SUPERMERCADO PASTORINHO           SUPERMERCADO           SUPERMERCADO   1.000000

Relatório de Classificação:
   

In [57]:
model.fit(X, y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=42))])

In [58]:
df = pd.read_excel('raw_teste_unique.xlsx')
df = df[~df['Descricao'].str.contains(r'\bPIX\b', regex=True)]
df['Descricao'] = df['Descricao'].str.findall(r'\b(?!\d+\b)\w{2,}\b').str.join(' ').str.upper().str.strip()
for w in sujeira_prefixos + sujeira_sufixos:
    df['Descricao'] = df['Descricao'].str.replace(w, '')

y_pred = model.predict(df['Descricao'])

df_final = pd.DataFrame({
    'Descricao': df['Descricao'],
    'Real': df['Categoria'],
    'Previsto': y_pred
})

probas = model.predict_proba(df['Descricao'])
max_probas = np.max(probas, axis=1)
df_final['Confianca'] = max_probas

print(df_final.head(10))

print("\nRelatório de Classificação:")
print(classification_report(df['Categoria'], y_pred))

                              Descricao          Real  Previsto  Confianca
0   RENDIMENTOS POUP FACIL DEPOS PARTIR   RENDIMENTOS     TAXAS   0.430636
1                        CASA DAS CORES    CONSTRUÇÃO  IMPOSTOS   0.550000
2                  RODOVIA BANDEIRANTES    TRANSPORTE     TAXAS   0.430636
3                  PAR DE NSRA APARECID        DOAÇÃO       GAS   0.250000
4                          MEUMARKET24H  SUPERMERCADO     TAXAS   0.430636
5                     GOOD BOM LOJA HOR  SUPERMERCADO    ROUPAS   0.990000
6                          MEUMARKET24H  SUPERMERCADO     TAXAS   0.430636
9                          POGERE FRIOS  SUPERMERCADO     TAXAS   0.430636
10                    MARAVILHAS DO LAR          CASA      AGUA   0.320000
11                         SAM CAMPINAS  SUPERMERCADO     TAXAS   0.430636

Relatório de Classificação:
                       precision    recall  f1-score   support

                 AGUA       0.00      0.00      0.00         0
                 CA

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_